# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121700e+02     1.496662e+00
 * time: 0.1412220001220703
     1     1.104301e+01     9.903399e-01
 * time: 1.0294218063354492
     2    -1.176435e+01     9.992771e-01
 * time: 1.165496826171875
     3    -3.396556e+01     7.519891e-01
 * time: 1.334500789642334
     4    -4.765667e+01     5.654005e-01
 * time: 1.4911727905273438
     5    -5.699939e+01     2.020928e-01
 * time: 1.6357169151306152
     6    -5.983815e+01     1.390871e-01
 * time: 1.7550909519195557
     7    -6.087507e+01     5.408502e-02
 * time: 1.8866028785705566
     8    -6.126667e+01     8.383635e-02
 * time: 2.010502815246582
     9    -6.157529e+01     4.073777e-02
 * time: 2.1234498023986816
    10    -6.179255e+01     2.988812e-02
 * time: 2.2450218200683594
    11    -6.196652e+01     2.548938e-02
 * time: 2.358613967895508
    12    -6.203639e+01     2.006003e-02
 * time: 2.4785878658294678
    13    -6.210793e+01     1.508076e-02
 * time: 2.59152793884277

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671072
    AtomicLocal         -18.8557686
    AtomicNonlocal      14.8522647
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336822

    total               -62.261666455829
